In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# dataloaders

from torchvision import transforms
from torchvision.datasets import EMNIST



BATCH_SIZE = 64
DATASET_ROOT_DIR = "./EMNIST_data"
SPLIT_TYPE = "balanced"


TRANSFORMATIONS = transforms.Compose(
    [
        transforms.RandomAffine(degrees=30, translate=(0.1, 0.2), scale=(0.5, 1)),
        transforms.RandomPerspective(distortion_scale=0.6, p=0.75),
        transforms.ToTensor(),
    ]
)


EMNIST_TRAIN, EMNIST_TEST = (
    EMNIST(root=DATASET_ROOT_DIR, split=SPLIT_TYPE, train=True, download=True, transform=transforms.ToTensor()),
    EMNIST(root=DATASET_ROOT_DIR, split=SPLIT_TYPE, train=False, download=True, transform=transforms.ToTensor()),
)


EMNIST_AUG_TRAIN, EMNIST_AUG_TEST = (
    EMNIST(root=DATASET_ROOT_DIR, split=SPLIT_TYPE, train=True, download=True, transform=TRANSFORMATIONS),
    EMNIST(root=DATASET_ROOT_DIR, split=SPLIT_TYPE, train=False, download=True, transform=TRANSFORMATIONS),
)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./EMNIST_data/EMNIST/raw/gzip.zip to ./EMNIST_data/EMNIST/raw


In [5]:
from google.colab import drive
drive.mount("mnt")

Mounted at mnt


In [6]:
%cd "mnt/My Drive/Colab Notebooks"

/content/mnt/My Drive/Colab Notebooks


In [9]:
#!pip install import-ipynb

In [10]:
import import_ipynb

import model1

importing Jupyter notebook from model1.ipynb


In [11]:
model = model1.InceptionModel(num_classes=47).to(device)

In [12]:
learning_rate = 1e-5
batch_size = BATCH_SIZE
epochs = 10

In [13]:
loss_fn = nn.CrossEntropyLoss()

In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
from tqdm import tqdm

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    for batch, (X, y) in tqdm(enumerate(dataloader)):
        X = X.to(device)
        y = y.to(device)

        get_pred = model(X)
        get_loss = loss_fn(get_pred, y)

        optimizer.zero_grad()
        get_loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            get_loss, current = get_loss.item(), batch * len(X)
            print(f"loss: {get_loss:>7f}  [{current}/{size}]")

In [17]:
def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, true_ = 0, 0

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            true_ += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    true_ /= size
    print(f"Accuracy: {(100*true_)}%")

In [22]:
#print(model)

# Train and test without an augmentation

In [18]:
train_loader = DataLoader(EMNIST_TRAIN, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(EMNIST_TEST, batch_size=BATCH_SIZE, shuffle=True)

In [19]:
!du -lh EMNIST_data/EMNIST/raw/gzip.zip

536M	EMNIST_data/EMNIST/raw/gzip.zip


In [20]:



epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n")
    train_loop(train_loader, model, loss_fn, optimizer, device)
    test_loop(test_loader, model, loss_fn, device)



Epoch 1



1it [00:07,  7.79s/it]

loss: 3.883924  [0/112800]


101it [00:24,  4.79it/s]

loss: 3.994988  [6400/112800]


...

loss: 3.743641  [102400/112800]


1701it [05:09,  4.56it/s]

loss: 3.812359  [108800/112800]


1763it [05:20,  5.50it/s]
100%|██████████| 294/294 [00:16<00:00, 17.58it/s]

Accuracy: 5.117021276595745%


In [21]:
#!nvidia-smi

# Train and test with an augmentation

In [22]:
train_loader1 = DataLoader(EMNIST_AUG_TRAIN, batch_size=BATCH_SIZE, shuffle=True)
test_loader1 = DataLoader(EMNIST_AUG_TEST, batch_size=BATCH_SIZE, shuffle=True)

In [23]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n")
    train_loop(train_loader1, model, loss_fn, optimizer, device)
    test_loop(test_loader1, model, loss_fn, device)


Epoch 1



1it [00:00,  2.86it/s]

loss: 3.913911  [0/112800]


101it [00:18,  4.49it/s]

loss: 3.781837  [6400/112800]


...

loss: 3.851154  [102400/112800]


1701it [05:08,  4.54it/s]

loss: 3.812246  [108800/112800]


1763it [05:20,  5.51it/s]
100%|██████████| 294/294 [00:22<00:00, 13.20it/s]

Accuracy: 3.7872340425531914%
